In [85]:
import numpy as np
from scipy.io import loadmat
import open3d
import scipy
from scipy.spatial import distance

def rigid_motion(p,q):
    """
    Least-Squares Rigid Motion Using Singular Value Decomposition. 
    (https://igl.ethz.ch/projects/ARAP/svd_rot.pdf) 
    
    (note: so far only for the easy case, where all weights are = 1)
    
    p,q: shape [num_points, 3]
    
    """
    n,d = p.shape
    
    # compute centroids
    p_cen = sum(p)/len(p)
    q_cen = sum(q)/len(q)
    
    # compute centered vectors
    X = np.array([i-p_cen for i in p])
    Y = np.array([i-q_cen for i in q])
    
    # compute covariance matrix 
    W = np.eye(n)
    S =  X.T.dot(W).dot(Y)
    
    # compute sigular value decomposition
    U, _, V = np.linalg.svd(S)
    
    # compute optimal R and t
    M = np.eye(d)
    M[-1,-1] = np.linalg.det(V.T.dot(U.T))
    R = V.T.dot(M).dot(U.T)
    
    t = q_cen - R.dot(p_cen)
    
    return R, t

def rms_error(p, q):
    n = p.shape[0]
    dist = [distance.euclidean(p[i,:], q[i,:]) for i in range(n)]
    return np.sqrt(np.sum(np.power(dist, 2))/n)

def show_fitting_result(a_1, a_2):
    
    pc1 = open3d.PointCloud()
    pc1.points = open3d.Vector3dVector(a_1)
    
    pc2 = open3d.PointCloud()
    pc2.points = open3d.Vector3dVector(a_2)

    open3d.draw_geometries([pc1, pc2])

def icp(a_1, a_2, convergence_treshold=0.005, point_selection="all", sample_size=500, generate_3d = True, verbose = True, is_test = False , accuracy_check = False, stability_constant = 1):
    """
    a_1: positions of points in point cloud 1. shape : [num_points1, 3]
    a_2: positions of points in point cloud 2. shape : [num_points2, 3]
    
    """
    
    if is_test:
        generate_3d = False
        verbose = False
    n,d = a_1.shape
    
    a_2_c = a_2.copy()
    
    # Filter the point clouds based on the depth,
    # only keep the indices where the z of the point cloud is less than 1
    valid_bool_1 = (a_1[:,2])<1
    valid_bool_2 = (a_2[:,2])<1
    a_1 = a_1[valid_bool_1]
    a_2 = a_2[valid_bool_2]
    
    # Point selection
    # Uniform subsampling
    if point_selection == "uniform":
        a_1 = a_1[np.random.randint(low=0, high=a_1.shape[0], size=sample_size)]
        a_2 = a_2[np.random.randint(low=0, high=a_2.shape[0], size=sample_size)]
    if stability_constant == 1 :
        R_overall = np.eye(d)
        t_overall = np.zeros(d)
    else:
        R_overall = np.random.normal(0,1, size = (d,d))*stability_constant
        t_overall = np.random.normal(0,1, size = d)*stability_constant
    
    # Base loop on difference in rsm error
    rms_error_old = 10000
    rms_error_new = rms_error_old-1
    
    while rms_error_old-rms_error_new > convergence_treshold:
        
        if point_selection == "random":
            a_1 = a_1[np.random.choice(a_1.shape[0], sample_size, replace=False), :]
            a_2 = a_2[np.random.choice(a_2.shape[0], sample_size, replace=False), :]
            
        # (Step 1) Find closest points for each point in a_1 from a_2
        tree = scipy.spatial.KDTree(a_2)
        closest_dists, closest_idx = tree.query(a_1)
        # Found this on stackoverflow: https://bit.ly/2P8IYiw
        # Not sure if we can use this, but it is definetely much (!!) faster 
        # than manually comparing all the vectors.
        # Usage also proposed on Wikipedia: https://bit.ly/2urg9nU
        # For how-to-use see: https://bit.ly/2UbKNfn
        closest_a_2 = a_2[closest_idx]
    
        # (Step 2) Refine R and t using Singular Value Decomposition
        R, t = rigid_motion(a_1,closest_a_2)
       
        # update a_1
        a_1 = np.array([R.dot(a)+t for a in a_1])
        
        # update rms error
        rms_error_old = rms_error_new
        rms_error_new = rms_error(a_1, closest_a_2)
        
        if verbose:
            print(rms_error_new)
        
        # update overall R and t
        R_overall = R.dot(R_overall)
        t_overall = R.dot(t_overall) + t
    if generate_3d:
        show_fitting_result(a_1, a_2_c)
    if accuracy_check:
        return rms_error_new
    return R_overall, t_overall
    

In [9]:
# print("Load a ply point cloud, print it, and render it")
# pcd1  = open3d.read_point_cloud("Data/data/0000000000.pcd")
# pcd2  = open3d.read_point_cloud("Data/data/0000000001.pcd")
# print(np.asarray(pcd.points))
# open3d.draw_geometries([pcd])
a_1 = loadmat("Data/source.mat")["source"].T
a_2 = loadmat("Data/target.mat")["target"].T

In [61]:
# test icp
icp(a_1 = a_1, a_2 = a_2, point_selection = 'random', is_test = True )

(array([[ 0.97710189,  0.01786619,  0.21202052],
        [-0.07518426,  0.96117657,  0.26549375],
        [-0.1990458 , -0.27535505,  0.94051069]]),
 array([-0.28350519,  0.23220625, -0.11435293]))

In [14]:
sampling_methods = ['random', 'uniform', 'default']

In [87]:
def speed_check(a_1, a_2, sampling_methods):
    for method in sampling_methods:
        print("Time that it takes for {} method is:".format(method))
        %time  icp(a_1, a_2, point_selection = method, is_test = True)
speed_check(a_1, a_2, sampling_methods)

Time that it takes for random method is:
CPU times: user 2.35 s, sys: 48.4 ms, total: 2.39 s
Wall time: 1.33 s
Time that it takes for uniform method is:
CPU times: user 2.22 s, sys: 39.6 ms, total: 2.25 s
Wall time: 1.23 s
Time that it takes for default method is:
CPU times: user 26.6 s, sys: 2.13 s, total: 28.8 s
Wall time: 25.2 s


In [86]:
def accuracy_check(a_1, a_2, sampling_methods):
    for method in sampling_methods:
        print("RMSE of {} method is {:.4}".format(method, icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True)))
accuracy_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.1419
RMSE of uniform method is 0.1359
RMSE of default method is 0.05849


In [84]:
def  noise_check(a_1, a_2, sampling_methods):
    noise_1, noise_2 = np.random.normal(0,1,(a_2.shape)) ,np.random.normal(0,1,(a_2.shape))
    a_1_noisy, a_2_noisy = a_1 + noise_1 , a_2 + noise_2
    for method in sampling_methods:
        rmse_normal =icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True) 
        rmse_noisy = icp(a_1_noisy, a_2_noisy, point_selection = method, is_test = True, accuracy_check = True)
        print("RMSE of {} method is {:.4}, whereas if we add noise, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_noisy, rmse_normal - rmse_noisy))
        R_normal , t_normal = icp(a_1, a_2, point_selection = method, is_test = True) 
        R_noisy , t_noisy = icp(a_1_noisy, a_2_noisy, point_selection = method, is_test = True) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_noisy), np.linalg.norm(t_normal - t_noisy)
        print("The distance between normal and noisy R matrices is {:.4} and between normal and noisy t vectors is {:.4}".format(R_distance, t_distance))
        

noise_check(a_1, a_2, sampling_methods)
    

RMSE of rand method is 0.1363, whereas if we add noise its 0.3933 , the difference is -0.257
The distance between R matrices is 0.4497 and between t vectors is 0.3504
RMSE of unif method is 0.1292, whereas if we add noise its 0.447 , the difference is -0.3177
The distance between R matrices is 0.4655 and between t vectors is 0.3769
RMSE of defa method is 0.05849, whereas if we add noise its 0.218 , the difference is -0.1595
The distance between R matrices is 0.4776 and between t vectors is 0.442


In [89]:
def  stability_check(a_1, a_2, sampling_methods):
    stability_constant = 10
    for method in sampling_methods:
        rmse_normal =icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True) 
        rmse_non_stable= icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True, stability_constant = stability_constant)
        print("RMSE of {} method is {:.4}, whereas if we have random initialisation, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_non_stable, rmse_normal - rmse_non_stable))
        R_normal , t_normal = icp(a_1, a_2, point_selection = method, is_test = True) 
        R_non_stable , t_non_stable = icp(a_1, a_2, point_selection = method, is_test = True, stability_constant = stability_constant) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_non_stable), np.linalg.norm(t_normal - t_non_stable)
        print("The distance between normal and randomely initialised R matrices is {:.4} and between normal and randomely initialised t vectors is {:.4}".format(R_distance, t_distance))
        

stability_check(a_1, a_2, sampling_methods)
    

RMSE of rand method is 0.122, whereas if we have random initialisation, it becomes 0.1384 , the difference is -0.01637
The distance between normal and randomely initialised R matrices is 34.02 and between normal and randomely initialised t vectors is 25.82
RMSE of unif method is 0.134, whereas if we have random initialisation, it becomes 0.1392 , the difference is -0.005192
The distance between normal and randomely initialised R matrices is 39.35 and between normal and randomely initialised t vectors is 9.126
RMSE of defa method is 0.05849, whereas if we have random initialisation, it becomes 0.05849 , the difference is 0.0
The distance between normal and randomely initialised R matrices is 29.26 and between normal and randomely initialised t vectors is 12.13


Note: Take stability as an example, an experimental setup may be the following: given a source point cloud, augmented target point cloud can be obtained by transforming the source point cloud using a random rigid transform R and t. Stability can be analyzed by observing a behaviour of ICP on source and augmented target point cloud w.r.t changes in magnitude of R and t.
        

- stability is about convergence of the algorithm dependent on initial condition.

- tolerance to noise is about convergence of the algorithm with input data with noise. You can imagine data is captured by a sensor. In the ideal case you will obtain exact point cloud, however sensor is not precise, therefore there will be noise in measurement. Therefore we ask you to evaluate how ICP is robust against those kind of issuses.